In [1]:
!pip install rfc3986

import pandas as pd
import re
from urllib.parse import urlparse
import urllib.robotparser #per gestire il file robot.txt
import xml.etree.ElementTree as ET  #per gestire i file xml
import requests #per http
import time #per gestire il tempo
from bs4 import BeautifulSoup #per gestire il parsing del html (si può usare anche per xml)
import rfc3986 # per la normalizzazione degli urls
import logging
import asyncio
import aiohttp

In [41]:

#url di prova
# ha sitemap-index ma non priority etc.
start_url_uno = "https://www.dragopublisher.com/it/"
# non ha sitemap
start_url_due = "https://www.wildraccoon.it/shop/"
# sito con sitemap
start_url = "https://www.bbc.com/"




# header generated by ChatGPT
useragent_dict_header = {'Googlebot':'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
                    'Bingbot':'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)',
                    'Slurp':'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)',
                    'DuckDuckbot':'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)',
                    'Yandex':'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)',
                    '*': 'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)'}


class UserAgentPolicy:
  '''
     Class to identify user agents and store relevant information to manage their requests appropriately over time
  '''
  def __init__(self, name, base_url, path_disallow, crawl_delay, request_rate,last_access, header, visited):
        self.lock = asyncio.Lock()
        self.base_url = base_url
        self.name = name
        self.path_disallow = path_disallow
        self.crawl_delay = crawl_delay
        self.request_rate = request_rate
        self.last_access = last_access
        self.header = header
        self.visited = visited


# Setup logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S'
)

logging.basicConfig(level=logging.DEBUG)


In [11]:
useragent_dict = {
    "Googlebot": UserAgentPolicy("Googlebot", None, None, None, None, 1e9,
                                 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)', set()),

    "Bingbot": UserAgentPolicy("Bingbot", None, None, None, None, 1e9,
                               'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)', set()),

    "Slurp": UserAgentPolicy("Slurp", None, None, None, None, 1e9,
                             'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)', set()),

    "DuckDuckbot": UserAgentPolicy("DuckDuckbot", None, None, None, None, 1e9,
                                   'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)', set()),

    "Yandex": UserAgentPolicy("Yandex", None, None, None, None, 1e9,
                              'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)', set()),

    "*": UserAgentPolicy("*", None, None, None, None, 1e9,
                         'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)', set())
}

default_agent=useragent_dict["Googlebot"]

#Fetcher#

In [57]:
from types import new_class
async def check_robots(url,useragent=default_agent):

  '''
    Inputs:
        url (str): The base URL of the website to analyze.
        useragent (class): The user agent to use when checking access permissions. Default is '*'.

    Output:
        sitemap (list or None): List of sitemap URLs found in robots.txt, if any.

    Description:
        This function checks the robots.txt file of a given website to determine:
        - Whether the given user agent is allowed to fetch content.
        - Which paths are disallowed for the user agent.
        - The crawl delay and request rate for the user agent.
        - Any sitemap URLs provided in the robots.txt.

        It updates the given useragent object with these details.
    '''

  useragent.base_url = url

  # Initialize the robot parser
  rfl = urllib.robotparser.RobotFileParser()
  # Construct the full URL to the robots.txt file
  new_url = url + "/robots.txt"

  try:
        async with aiohttp.ClientSession() as session:
            async with session.get(new_url, timeout=10) as response:
                if response.status == 200:
                    robots_txt = await response.text()
                    # Setup and parse with robotparser
                    rfl.set_url(new_url)
                    rfl.parse(robots_txt.splitlines())
                else:
                    logging.warning(f"Failed to fetch robots.txt: HTTP {response.status}")
                    return None
  except aiohttp.ClientError as e:
        logging.error(f"Error fetching robots.txt from {new_url}: {e}")
        return None


  # Check if the user agent is allowed to fetch its own robots.txt
  if rfl.can_fetch(useragent.name, new_url):
        logging.info(f"{useragent.name} is allowed to fetch {new_url}")
  else:
        logging.warning(f"{useragent.name} is NOT allowed to fetch {new_url}")

  # Check if the useragent can fetch the informations
  check = rfl.can_fetch(useragent.name,new_url)

  # Retrieve the list of the path not to follow
  path_disallow = rfl.parse("Disallow")
  useragent.path_disallow = path_disallow

  # Retrieve the crawl delay (minimum interval between requests)
  crawl_delay = rfl.crawl_delay(useragent.name)
  useragent.crawl_delay = crawl_delay

  # Retrieve the request rate (tuple of requests per time interval)
  request_rate = rfl.request_rate(useragent.name)
  useragent.request_rate = request_rate

  # Retrieve any sitemap URLs declared in the robots.txt
  sitemap = rfl.site_maps()

  return sitemap





In [14]:

async def check_time(useragent=default_agent):

  '''
    Inputs:
        useragent (UserAgentPolicy): The user agent object containing rate-limiting information.

    Output:
      None

    Description:
        Ensures that the user agent respects the specified crawl delay and request rate policies:
        - Calculates the appropriate delay based on the more restrictive value between `crawl_delay` and `request_rate`.
        - If neither is defined, defaults to a delay of 1.5 seconds.
        - Ensures the delay is respected by pausing execution as needed.
        - Updates `last_access` to the current time after waiting.
  '''


  # If both crawl_delay and request_rate are defined, use the more restrictive one (converted to nanoseconds)

  if useragent.request_rate and useragent.crawl_delay:
     # Calculate delay from request rate: total interval (in ns) divided by number of requests
     request_delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]
     # Convert crawl delay from seconds to nanoseconds
     crawl_delay = useragent.crawl_delay * 1e9
     # Use the larger (i.e., more restrictive) delay
     delay = max(request_delay, crawl_delay)

  elif useragent.crawl_delay:
     delay = useragent.crawl_delay * 1e9

  elif useragent.request_rate:
     delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]

  # If neither crawl_delay and request rate are defined, use a default delay of 1.5 seconds
  else:
    delay = 1.5 * 1e9  # 1.5 secondi in nanosecondi

  # Get the current time in nanoseconds
  now = time.monotonic_ns()
  wait = max(0, (useragent.last_access + delay) - now)

  if wait > 0:
     await asyncio.sleep(wait / 1e9)

  # Update last_access to the current time to enforce timing on next request
  useragent.last_access = time.monotonic_ns()


In [25]:
async def fetch(url, useragent=default_agent):
    '''
    Inputs:
        url (str): The target URL to fetch.
        useragent (UserAgentPolicy): The user agent object containing headers and rate-limiting rules.

    Outputs:
        str or None: The HTML content of the page if the request is successful (HTTP 200), otherwise None.

    Description:
        Sends an HTTP GET request to the specified URL, respecting the request policies defined for the given user agent.
    '''
    # To avoid simultaneous tasks between same useragents
    async with useragent.lock:

      # Check if the page has been visited already
      if url in useragent.visited:
          logging.warning("Page visited already")
          return None

      # Enforce crawl delay and request rate restrictions for the user agent
      await check_time(useragent)

      headers = {
          'User-Agent': useragent.header
      }

      try:
          async with aiohttp.ClientSession() as session:
              async with session.get(url, headers=headers, timeout=10) as response:
                  if response.status == 200:
                      html = await response.text()
                      logging.info("Page successfully recovered")
                      useragent.visited.add(url)
                      return html
                  else:
                      logging.warning(f"Page not available: {url} (Status {response.status})")
                      return None
      except aiohttp.ClientError as e:
          logging.error(f"Error, impossible to fetch {url}: {e}")
          return None


#Parser#

In [19]:
def normalize_url(url):

  '''
    Input:
        url (str): The URL to be normalized.

    Output:
        str: The normalized URL as a string. If normalization fails, the original URL is returned.

    Description:
        Normalizes the input URL according to the RFC 3986 standard and returns a string representation.
        This function uses the `rfc3986` library to parse and normalize the URL according to the
        URI standard defined in RFC 3986. This includes handling issues such as case normalization,
        removing default ports, sorting query parameters (if applicable), and more.

        If an error occurs during normalization (e.g., invalid input), it catches the exception
        and returns the original URL as a fallback.
  '''

  try:
      uri = rfc3986.uri_reference(url).normalize()
      return uri.unsplit()

  except Exception as e:
      logger.error("Error, impossble to normalize %s: %s", url, e)
      return url

In [18]:
def check_spider_traps(url):

  '''

    Input:
     url(str):The URL to be analyzed.

    Output:
      - Returns False if the URL is suspicious or considered a "spider trap."
      - Returns True if the URL seems safe to crawl.

    Description:
      Checks whether a given URL is potentially harmful or could trap a web crawler
      in infinite loops or unnecessary crawling paths.

  '''

  MAX_URL_LENGTH = 200   # Arbitrary maximum allowed URL length
  MAX_PATH_DEPTH = 6     # Maximum allowed number of slashes in path
  trap_pattern = re.compile(r"(calendar|sessionid|track|ref|sort|date=|page=\d{3,})", re.IGNORECASE)   # Pattern matching common signs of spider traps:
                                                                                                       # calendars, session IDs, tracking params, endless pagination, etc.

  link = urlparse(url)

  # Check URL length
  if len(link) > MAX_URL_LENGTH:
     return False

  # Check path depth (number of '/' in path)
  if link.path.count('/') > MAX_PATH_DEPTH:
     return False

  # Check for suspicious patterns in the URL
  if trap_pattern.search(url):
     return False

  return True




In [20]:



async def fetch_sitemap(session, sitemap_url):
    '''
    Input:
    session : aiohttp.ClientSession
        An existing aiohttp session used to make the HTTP request.

    sitemap_url : str
        The URL pointing to a sitemap (typically found in robots.txt or known ahead of time).

    Output:
    list of dict
        A list of dictionaries containing:
            - 'url': the page URL
            - 'priority': optional priority value (from XML)
            - 'update': optional change frequency (from XML)
        If the sitemap is invalid or inaccessible, returns an empty list.

    Description:
    Asynchronously fetches and parses a single sitemap (either XML or HTML) and returns a list of URLs of sitemaps
    This function supports:
        - XML sitemaps (including recursive sitemap index support)
        - HTML sitemaps (basic <a> link extraction)
    It validates HTTP status, parses content based on MIME type, and returns structured data
    ready for further processing or filtering.
    '''

    try:
        # Perform an asynchronous HTTP GET request for the sitemap URL
        async with session.get(sitemap_url, timeout=10) as response:

            # Handle HTTP failure (non-200 status)
            if response.status != 200:
                logger.warning(f"Failed to fetch {sitemap_url} (Status {response.status})")
                return []

            # Extract MIME type and content
            content_type = response.headers.get("Content-Type", "").lower()
            content = await response.text()

            # XML Sitemap
            if "xml" in content_type:
                try:
                    # Parse the XML content into an ElementTree structure
                    root = ET.fromstring(content.encode())
                except ET.ParseError as e:
                    logger.error(f"XML parse error in {sitemap_url}: {e}")
                    return []

                # Define the XML namespace used by standard sitemaps
                namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

                # Handle <sitemapindex> files (recursive fetching of sub-sitemaps)
                if root.tag.endswith('index'):
                    sub_sitemaps = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
                    results = []
                    for sub_url in sub_sitemaps:
                        # Recursively fetch and parse each child sitemap
                        results.extend(await fetch_sitemap(session, sub_url))
                    return results

                # Handle regular <urlset> sitemap with actual page URLs
                elif root.tag.endswith('urlset'):
                    return [{
                        'url': url.findtext('ns:loc', default='', namespaces=namespace),
                        'priority': url.findtext('ns:priority', default=None, namespaces=namespace),
                        'update': url.findtext('ns:changefreq', default=None, namespaces=namespace)
                    } for url in root.findall('ns:url', namespace)]

            #HTML Sitemap

            elif "html" in content_type:
                soup = BeautifulSoup(content, "html.parser")
                return [{
                    'url': normalize_url(a['href']),  # Normalize the extracted link
                    'priority': None,                 # HTML sitemaps usually don't include priority
                    'update': None
                } for a in soup.find_all('a', href=True)]

            # Unsupported format
            else:
                logger.warning(f"Unsupported format: {sitemap_url} ({content_type})")
                return []

    # Handle network or HTTP client exceptions
    except aiohttp.ClientError as e:
        logger.error(f"Error fetching {sitemap_url}: {e}")
        return []


async def parse_sitemap(sitemap_list):
    '''
    Input:
        sitemap_list (list): A list of sitemap URLs to parse.

    Output:
          Returns a DataFrame containing:
         - URL
         - Priority
         - Update frequency


    Description:
         This function uses aiohttp to fetch sitemaps asynchronously. It supports:
            - XML sitemaps (standard or sitemap index)
            - HTML-based sitemaps as fallback
        It normalizes the URLs, filters out potential spider traps,
        and merges the results into a clean DataFrame for crawling or analysis.
    '''

    # If the list is empty, return an empty DataFrame
    if not sitemap_list:
        logger.warning("No sitemap available.")
        return pd.DataFrame(columns=['url', 'priority', 'update'])

    all_entries = []  # Accumulate all parsed and cleaned URL entries here

    # Create an asynchronous HTTP session for efficient reuse of connections
    async with aiohttp.ClientSession() as session:

        # Create a list of async tasks for all sitemap URLs to fetch and parse them concurrently
        tasks = [fetch_sitemap(session, sitemap_url) for sitemap_url in sitemap_list if sitemap_url]

        # Run all fetch tasks concurrently and wait for them to complete
        results = await asyncio.gather(*tasks)

        # Process each list of entries returned from individual sitemap URLs
        for entries in results:
            for entry in entries:
                url = normalize_url(entry['url'])  # Normalize the URL to a standard format
                if check_spider_traps(url):        # Filter out known spider traps
                    entry['url'] = url
                    all_entries.append(entry)       # Keep the cleaned and validated entry

    # Convert list of entries into a DataFrame and remove duplicate URLs
    df = pd.DataFrame(all_entries, columns=['url', 'priority', 'update']).drop_duplicates()

    return df



def parse_page_url(html,sitemaps_urls,useragent=default_agent):

  '''
     Inputs:
     - html(str): The HTML content of the page.
     - sitemaps_urls(str): A list of URLs already known from sitemaps.
     - useragent(class): A user agent object containing disallowed paths (robots.txt rules).

     Output:
     - A list of new, allowed URLs extracted from the page.

     Description:
       Given an HTML page, this function extracts all URLs and filters them based on:
       - Exclusion of private or disallowed directories (from robots.txt rules)
       - Deduplication with URLs already found in the sitemap
       - Optional normalization and trap filtering

  '''

  # Parse HTML content
  soup = BeautifulSoup(html, 'html.parser')

  urls = []
  # Extract URLs from href attributes (e.g., <a>, <link>, etc.)
  for tag in soup.find_all(href=True):
      urls.append(tag['href'])

  # Extract URLs from src attributes (e.g., <img>, <script>, etc.)
  for tag in soup.find_all(src=True):
      urls.append(tag['src'])

  # Filter out disallowed URLs based on robots.txt rules
  if useragent.path_disallow != None:
    for url in urls:
      for path in useragent.path_disallow:
        if path in url:
         urls.remove(url)

  # Normalize URLs (e.g., remove fragments, resolve relative paths, etc.)
    urls = [normalize_url(url) for url in urls]

  # Remove duplicate URLs
    urls = list(set(urls))

  # Filter out potential spider traps
    urls = [url for url in urls if check_spider_traps(url)]

  # Filter out URLs that are already in the sitemap
  if sitemaps_urls != None:
    def_urls = sitemaps_urls
    for url in urls:
      if url not in def_urls:
         def_urls.append(url)
  else:
    def_urls = urls

  return def_urls




In [17]:

def parse_page_tags_all(html,tags_type = None):

  '''
    Inputs:
    - html: The HTML content as a string.
    - tags_type: A list of tag names to search for (default includes common content tags).

    Output:
    - A list of text strings extracted from the specified tags, preserving DOM order

    Description:
      Parses the given HTML content and extracts text from specified HTML tags.

  '''
  if tags_type == None:
     tags_type = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'span', 'a']

  # Initialize the HTML parser
  soup = BeautifulSoup(html, "html.parser")

  # Find tag to remove
  for tag in soup(['script','style','footer','nav','noscript','header','form','aside']):
      tag.decompose()

  # Find all comments and removes them
  for comment in soup.find_all(string=lambda text: isinstance(text,comment)):
      comment.extract()

  # Find all tags of the specified types (respects DOM order)
  tags = soup.find_all(tags_type)

  # Extract clean text from each tag (removing whitespace and combining with spaces)
  texts = [tag.get_text(separator=' ', strip=True) for tag in tags]

  return texts




#Test#

In [55]:

async def main_page(start_url: str):
    # STEP 1: Recupera robots.txt e ottieni le sitemap (se presenti)
    sitemaps = await check_robots(start_url, useragent=default_agent)

    if not sitemaps:
        print(f"[!] Nessuna sitemap trovata per {start_url}")
        return

    print(f"[✓] Sitemap trovate: {sitemaps}")

    # STEP 2: Parsing asincrono delle sitemap
    df = await parse_sitemap(sitemaps)
    print(f"[✓] URLs raccolti dalle sitemap: {len(df)}")

    if df.empty:
        print("[-] Nessun URL valido trovato nella sitemap.")
        return

    # STEP 3: Scegli un URL da testare (il primo della sitemap)
    test_url = df.iloc[0]["url"]
    print(f"[→] Provo a fare fetch della pagina: {test_url}")

    # STEP 4: Recupera HTML
    html = await fetch(test_url, useragent=default_agent)
    if html is None:
        print("[-] Impossibile scaricare la pagina.")
        return

    # STEP 5: Estrai contenuti testuali dalla pagina
    texts = parse_page_tags_all(html)
    print(f"[✓] Estratti {len(texts)} blocchi di testo.")

    # STEP 6: Estrai e filtra nuovi link dalla pagina
    new_links = parse_page_url(html, sitemaps_urls=list(df['url']), useragent=default_agent)
    print(f"[✓] Nuovi link scoperti e filtrati: {len(new_links)}")

    # Output di esempio
    print("\n[+] Esempio di testo trovato:")
    for t in texts[:3]:
        print(f"- {t}")

    print("\n[+] Esempio di link nuovi:")
    for link in new_links[:5]:
        print(f"- {link}")


In [ ]:
await main_page(start_url)

[✓] Sitemap trovate: ['https://www.bbc.com/sitemaps/https-index-com-archive.xml', 'https://www.bbc.com/sitemaps/https-index-com-news.xml', 'https://www.bbc.com/sitemaps/https-index-com-archive_video.xml', 'https://www.bbc.com/sitemaps/https-index-com-video.xml', 'https://www.bbc.com/sitemaps/sitemap-com-ws-topics.xml', 'https://www.bbc.com/sport/sitemap.xml', 'https://www.bbc.com/sitemaps/sitemap-com-ws-topics.xml', 'https://www.bbc.com/afrique/sitemap.xml', 'https://www.bbc.com/arabic/sitemap.xml', 'https://www.bbc.com/bengali/sitemap.xml', 'https://www.bbc.com/burmese/sitemap.xml', 'https://www.bbc.com/gahuza/sitemap.xml', 'https://www.bbc.com/hausa/sitemap.xml', 'https://www.bbc.com/hindi/sitemap.xml', 'https://www.bbc.com/indonesia/sitemap.xml', 'https://www.bbc.com/mundo/sitemap.xml', 'https://www.bbc.com/pashto/sitemap.xml', 'https://www.bbc.com/persian/sitemap.xml', 'https://www.bbc.com/portuguese/sitemap.xml', 'https://www.bbc.com/russian/sitemap.xml', 'https://www.bbc.com/swah

#test più richieste stesso useragent

In [53]:
from datetime import datetime
import asyncio

async def fetch_simulated(url, useragent):
    print(useragent.last_access,useragent.crawl_delay,useragent.request_rate)
    await check_time(useragent)
    now = datetime.now().strftime('%H:%M:%S.%f')[:-3]
    print(f"[{useragent.name}] FETCH: {url} at {now}")
    await asyncio.sleep(0.1)  # simula tempo di elaborazione
    return f"Simulated HTML content from {url}"

async def main():
    url = start_url
    tasks = [fetch_simulated(url, default_agent) for _ in range(5)]
    await asyncio.gather(*tasks)


In [54]:
await main()

5544373121139 None None
[Googlebot] FETCH: https://www.bbc.com/ at 15:49:59.728
5567620365597 None None
5567620365597 None None
5567620365597 None None
5567620365597 None None
[Googlebot] FETCH: https://www.bbc.com/ at 15:50:01.232
[Googlebot] FETCH: https://www.bbc.com/ at 15:50:01.232
[Googlebot] FETCH: https://www.bbc.com/ at 15:50:01.232
[Googlebot] FETCH: https://www.bbc.com/ at 15:50:01.232
